In [1]:
import os
import pandas as pd
import altair as alt

# Aquisição e Limpeza de dados

Esse notebook documenta o processamento dos datasets baixados do INMET, com os agrupamentos necessários pra criar os arquivos `dados.csv`, `completo.csv` e `metadados.csv`.

Não é necessário para o resto do curso.

Baixe o arquivo do site [Dados históricos do INMET](https://portal.inmet.gov.br/dadoshistoricos), selecionando o ano de 2019 todo.

Descompacte em uma pasta, seguindo a estrutura:
``` bash
- baixado
--- extraído
```

Estamos prontos para ver os arquivos, mas mesmo sem abrir eles podemos ter alguma informação sobre esses dados.

Vamos listar todos os arquivos e ver quantas cidades tem por estado.

In [2]:
prefixo = "baixado/descompactado/"
nome_arquivos = os.listdir(prefixo)
print(nome_arquivos[:4])

['INMET_CO_DF_A001_BRASILIA_01-01-2019_A_31-12-2019.CSV', 'INMET_CO_DF_A042_BRAZLANDIA_01-01-2019_A_31-12-2019.CSV', 'INMET_CO_DF_A045_AGUAS EMENDADAS_01-01-2019_A_31-12-2019.CSV', 'INMET_CO_DF_A046_GAMA (PONTE ALTA)_01-01-2019_A_31-12-2019.CSV']


In [3]:
siglas_estado = [nome.split("_") for nome in nome_arquivos]
print(siglas_estado[:2]) # tem mais informações que só as iniciais
siglas_estado = [nome[2] for nome in siglas_estado]
print(siglas_estado[:20]) # agora está certo
siglas = list(set(siglas_estado))
siglas.sort()
print(siglas) # todos os estados estão presentes?

[['INMET', 'CO', 'DF', 'A001', 'BRASILIA', '01-01-2019', 'A', '31-12-2019.CSV'], ['INMET', 'CO', 'DF', 'A042', 'BRAZLANDIA', '01-01-2019', 'A', '31-12-2019.CSV']]
['DF', 'DF', 'DF', 'DF', 'DF', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO', 'GO']
['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']


In [4]:
dados_estado = []
for s in siglas:
    dados_estado.append({"estado":s, "cidades": siglas_estado.count(s)})
dataframe = pd.DataFrame(dados_estado)
dataframe

,estado,cidades
0,AC,7
1,AL,7
2,AM,19
3,AP,4
4,BA,47
5,CE,16
6,DF,5
7,ES,13
8,GO,26
9,MA,17


A tabela não ajuda muito a ver a distribuição, é melhor com um gráfico

In [5]:
alt.Chart(dataframe).mark_bar().encode(
    x="estado",
    y="cidades"
)

alt.Chart(...)

Dá pra ver melhor a disparidade, mas no nosso caso queremos só o Pará.  
Vamos ler um arquivo com a sigla `PA` e ver como está o formato.

In [6]:
from pprint import pprint
with open(prefixo + nome_arquivos[0], "r") as arquivo:
    pprint(arquivo.readlines()[:20])

['REGI?O:;CO\n',
 'UF:;DF\n',
 'ESTAC?O:;BRASILIA\n',
 'CODIGO (WMO):;A001\n',
 'LATITUDE:;-15,789343\n',
 'LONGITUDE:;-47,925756\n',
 'ALTITUDE:;1160,96\n',
 'DATA DE FUNDAC?O:;07/05/00\n',
 'Data;Hora UTC;PRECIPITAÇÃO TOTAL, HORÁRIO (mm);PRESSAO ATMOSFERICA AO NIVEL '
 'DA ESTACAO, HORARIA (mB);PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) '
 '(mB);PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB);RADIACAO GLOBAL '
 '(W/m²);TEMPERATURA DO AR - BULBO SECO, HORARIA (°C);TEMPERATURA DO PONTO DE '
 'ORVALHO (°C);TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C);TEMPERATURA MÍNIMA '
 'NA HORA ANT. (AUT) (°C);TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) '
 '(°C);TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C);UMIDADE REL. MAX. NA '
 'HORA ANT. (AUT) (%);UMIDADE REL. MIN. NA HORA ANT. (AUT) (%);UMIDADE '
 'RELATIVA DO AR, HORARIA (%);VENTO, DIREÇÃO HORARIA (gr) (° (gr));VENTO, '
 'RAJADA MAXIMA (m/s);VENTO, VELOCIDADE HORARIA (m/s);\n',
 '2019/01/01;0000 '
 'UTC;1;887;887;886,6;;18,5;17;18,7;18,4;17,3

Tem muita informação nas primeiras linhas do que a gente precisa, então vamos dividir: um arquivo de metadados e um arquivo com a informação dos sensores. Vamos começar adicionando os cabeçalhos.

In [7]:
dados = "Cidade,"
metadados = ""

with open(prefixo + nome_arquivos[0], "r", encoding="latin-1") as arquivo:
    linhas = arquivo.readlines()
    dados += linhas[8][:-2].replace(","," |").replace(";",",") + "\n"
    metadados = ",".join([linha.split(":;")[0].replace("?","Ã").replace("CÃ","ÇÃ") for linha in linhas[:8]]) + "\n"
    print(metadados)
    print(dados)

REGIÃO,UF,ESTAÇÃO,CODIGO (WMO),LATITUDE,LONGITUDE,ALTITUDE,DATA DE FUNDAÇÃO

Cidade,Data,Hora UTC,PRECIPITAÇÃO TOTAL | HORÁRIO (mm),PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO | HORARIA (mB),PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (W/m²),TEMPERATURA DO AR - BULBO SECO | HORARIA (°C),TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),UMIDADE RELATIVA DO AR | HORARIA (%),VENTO | DIREÇÃO HORARIA (gr) (° (gr)),VENTO | RAJADA MAXIMA (m/s),VENTO | VELOCIDADE HORARIA (m/s)



O cabeçalho dos dados ficou com muita informação extra, e um formato não tão agradável. Trocar o nome das colunas é uma função comum em vários processos de visualização, e será feito manualmente agora. Mas vamos salvar uma estrutura de chave valor, já que em vários títulos também tem informação sobre as medidas.

In [8]:
dados1 = "Cidade,Data,Hora,Precipitação,Pressão Atmosférica ao nível da estação,\
Pressão Atmosférica máxima,Pressão Atmosférica mínima,Radiação Global,Temperatura do ar - bulbo seco,Temperatura do ponto de orvalho,\
Temperatura máxima,Temperatura mínima,Temperatura orvalho máxima, Temperatura orvalho mínima,\
Umidade Relativa máxima,Umidade Relativa mínima,Umidade Relativa do Ar,Direção Horária do Vento,Rajada Máxima de Vento,Velocidade Horária do Vento"

cabecalho = {}
for d,d1 in zip(dados.split(","),dados1.split(",")):
    cabecalho[d1] = d

print(cabecalho)
import json

with open("cabecalho.json", "w+", encoding="utf-8") as arquivo:
    arquivo.write(json.dumps(cabecalho))

dados = dados1 + "\n"

{'Cidade': 'Cidade', 'Data': 'Data', 'Hora': 'Hora UTC', 'Precipitação': 'PRECIPITAÇÃO TOTAL | HORÁRIO (mm)', 'Pressão Atmosférica ao nível da estação': 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO | HORARIA (mB)', 'Pressão Atmosférica máxima': 'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)', 'Pressão Atmosférica mínima': 'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)', 'Radiação Global': 'RADIACAO GLOBAL (W/m²)', 'Temperatura do ar - bulbo seco': 'TEMPERATURA DO AR - BULBO SECO | HORARIA (°C)', 'Temperatura do ponto de orvalho': 'TEMPERATURA DO PONTO DE ORVALHO (°C)', 'Temperatura máxima': 'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)', 'Temperatura mínima': 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)', 'Temperatura orvalho máxima': 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)', ' Temperatura orvalho mínima': 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)', 'Umidade Relativa máxima': 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)', 'Umidade Relativa mínima': 'UMIDADE REL. MIN. NA HO

Vamos adicionar os dados e salvar os arquivos.

In [9]:
def extrair_metadados(linhas):
    return ",".join([linha.split(":;")[1].replace("\n","").replace(",",".") for linha in linhas[:8]]) + "\n"

for caminho, sigla in zip([prefixo + n for n in nome_arquivos], siglas_estado):
    if sigla == "PA":
        with open(caminho, "r", encoding="latin-1") as arquivo:
            linhas = arquivo.readlines()
            metadados += extrair_metadados(linhas)
            linhas_dados = [caminho.split("_")[4] + "," + linha.replace(",",".").replace(";",",")[:-2] + "\n" for linha in linhas[9:]]
            dados += "".join(linhas_dados)

Agora vamos salvar os dados completos e metadados.

In [10]:
with open("completo.csv", "w+", encoding="utf-8") as arquivo:
    arquivo.write(dados)

with open("metadados.csv", "w+", encoding="utf-8") as arquivo:
    arquivo.write(metadados)

Por fim vamos agrupar pela média de medições pelo dia para criar um arquivo mais leve.

In [11]:
df = pd.read_csv('completo.csv')
df["Data"] = pd.to_datetime(df["Data"], infer_datetime_format=True)  
df_leve = df.groupby(["Cidade",pd.Grouper(key='Data', freq='W')]).mean()
df_leve.to_csv("dados.csv")

Tudo pronto! Só seguir para a primeira aula agora.